In [1]:
import transformer_cloner as tc

print(tc.__version__)



0.2.0


In [ ]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer("google/embeddinggemma-300m")

sentences = [
    "That is a happy person",
    "That is a happy dog",
    "That is a very happy person",
    "Today is a sunny day"
]
embeddings = model.encode(sentences)

similarities = model.similarity(embeddings, embeddings)
print(similarities.shape)
# [4, 4]

You are trying to use a model that was created with Sentence Transformers version 5.1.0, but you're currently using version 4.1.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.


model.safetensors:   0%|          | 0.00/1.21G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/314 [00:00<?, ?it/s]

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

torch.Size([4, 4])


In [5]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 2048, 'do_lower_case': False}) with Transformer model: Gemma3TextModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Dense({'in_features': 768, 'out_features': 3072, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (3): Dense({'in_features': 3072, 'out_features': 768, 'bias': False, 'activation_function': 'torch.nn.modules.linear.Identity'})
  (4): Normalize()
)

In [3]:
# Load model directly
from transformers import AutoProcessor, AutoModelForSeq2SeqLM

processor = AutoProcessor.from_pretrained("google/t5gemma-2-270m-270m")
model = AutoModelForSeq2SeqLM.from_pretrained("google/t5gemma-2-270m-270m")

processor_config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


tokenizer_config.json:   0%|          | 0.00/830 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/6.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.57G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/911 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/195 [00:00<?, ?B/s]

In [2]:
from sentence_transformers import SentenceTransformer

def get_detailed_instruct(task_description: str, query: str) -> str:
    return f'Instruct: {task_description}\nQuery: {query}'

# Görev: Web arama sorgusuna uygun bilgiyi içeren pasajları getir
task = 'Given a Turkish search query, retrieve relevant passages written in Turkish that best answer the query'

queries = [
    get_detailed_instruct(task, 'Kolay bir kahvaltı tarifi nedir?'),
    get_detailed_instruct(task, 'Dış mekan yürüyüşü için en iyi saat hangisidir?')
]

documents = [
    "Güne enerjik başlamak için yulaf ezmesi, süt ve meyveyle hazırlanan basit bir kahvaltı hem pratik hem de besleyicidir. Üzerine biraz bal ve tarçın eklerseniz lezzeti artar.",
    "Sabah saatleri, özellikle 07:00 ile 10:00 arası, açık havada yürüyüş yapmak için idealdir. Bu saatlerde hava daha serin ve temiz olur, ayrıca gün ışığı vücut ritmini destekler.",
    "Türkiye'nin en uzun nehri Kızılırmak'tır. Sivas'tan doğar, Karadeniz'e dökülür ve yaklaşık 1.355 kilometre uzunluğundadır."
]

input_texts = queries + documents

model = SentenceTransformer('ytu-ce-cosmos/turkish-e5-large')

embeddings = model.encode(input_texts, convert_to_tensor=True, normalize_embeddings=True)
scores = (embeddings[:2] @ embeddings[2:].T) * 100

for i, query in enumerate(queries):
    print(f"\nSorgu: {query.split('Query: ')[-1]}")
    for j, doc in enumerate(documents):
        print(f"   → Belge {j+1} Skoru: {scores[i][j]:.2f}")
        print(f"     İçerik: {doc[:80]}...")

"""
Sorgu: Kolay bir kahvaltı tarifi nedir?
   → Belge 1 Skoru: 67.36
     İçerik: Güne enerjik başlamak için yulaf ezmesi, süt ve meyveyle hazırlanan basit bir ka...
   → Belge 2 Skoru: 31.68
     İçerik: Sabah saatleri, özellikle 07:00 ile 10:00 arası, açık havada yürüyüş yapmak için...
   → Belge 3 Skoru: 7.06
     İçerik: Türkiye'nin en uzun nehri Kızılırmak'tır. Sivas'tan doğar, Karadeniz'e dökülür v...

Sorgu: Dış mekan yürüyüşü için en iyi saat hangisidir?
   → Belge 1 Skoru: 28.14
     İçerik: Güne enerjik başlamak için yulaf ezmesi, süt ve meyveyle hazırlanan basit bir ka...
   → Belge 2 Skoru: 78.02
     İçerik: Sabah saatleri, özellikle 07:00 ile 10:00 arası, açık havada yürüyüş yapmak için...
   → Belge 3 Skoru: 18.70
     İçerik: Türkiye'nin en uzun nehri Kızılırmak'tır. Sivas'tan doğar, Karadeniz'e dökülür v...
"""


model.safetensors:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/391 [00:00<?, ?it/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]


Sorgu: Kolay bir kahvaltı tarifi nedir?
   → Belge 1 Skoru: 67.36
     İçerik: Güne enerjik başlamak için yulaf ezmesi, süt ve meyveyle hazırlanan basit bir ka...
   → Belge 2 Skoru: 31.68
     İçerik: Sabah saatleri, özellikle 07:00 ile 10:00 arası, açık havada yürüyüş yapmak için...
   → Belge 3 Skoru: 7.06
     İçerik: Türkiye'nin en uzun nehri Kızılırmak'tır. Sivas'tan doğar, Karadeniz'e dökülür v...

Sorgu: Dış mekan yürüyüşü için en iyi saat hangisidir?
   → Belge 1 Skoru: 28.14
     İçerik: Güne enerjik başlamak için yulaf ezmesi, süt ve meyveyle hazırlanan basit bir ka...
   → Belge 2 Skoru: 78.02
     İçerik: Sabah saatleri, özellikle 07:00 ile 10:00 arası, açık havada yürüyüş yapmak için...
   → Belge 3 Skoru: 18.70
     İçerik: Türkiye'nin en uzun nehri Kızılırmak'tır. Sivas'tan doğar, Karadeniz'e dökülür v...


"\nSorgu: Kolay bir kahvaltı tarifi nedir?\n   → Belge 1 Skoru: 67.36\n     İçerik: Güne enerjik başlamak için yulaf ezmesi, süt ve meyveyle hazırlanan basit bir ka...\n   → Belge 2 Skoru: 31.68\n     İçerik: Sabah saatleri, özellikle 07:00 ile 10:00 arası, açık havada yürüyüş yapmak için...\n   → Belge 3 Skoru: 7.06\n     İçerik: Türkiye'nin en uzun nehri Kızılırmak'tır. Sivas'tan doğar, Karadeniz'e dökülür v...\n\nSorgu: Dış mekan yürüyüşü için en iyi saat hangisidir?\n   → Belge 1 Skoru: 28.14\n     İçerik: Güne enerjik başlamak için yulaf ezmesi, süt ve meyveyle hazırlanan basit bir ka...\n   → Belge 2 Skoru: 78.02\n     İçerik: Sabah saatleri, özellikle 07:00 ile 10:00 arası, açık havada yürüyüş yapmak için...\n   → Belge 3 Skoru: 18.70\n     İçerik: Türkiye'nin en uzun nehri Kızılırmak'tır. Sivas'tan doğar, Karadeniz'e dökülür v...\n"